In [ ]:
# Install openvino package
%pip install -q "openvino>=2023.1.0"

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import openvino as ov

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import segmentation_map_to_image, download_file

In [ ]:
from pathlib import Path

base_model_dir = Path("./model").expanduser()

model_name = "semantic-segmentation-adas-0001"
model_xml_name = f'{model_name}.xml'
model_bin_name = f'{model_name}.bin'

model_xml_path = base_model_dir / model_xml_name

if not model_xml_path.exists():
    model_xml_url = "https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/road-segmentation-adas-0001/FP16/road-segmentation-adas-0001.xml"
    model_bin_url = "https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/road-segmentation-adas-0001/FP16/road-segmentation-adas-0001.bin"

    download_file(model_xml_url, model_xml_name, base_model_dir)
    download_file(model_bin_url, model_bin_name, base_model_dir)
else:
    print(f'{model_name} already downloaded to {base_model_dir}')

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
core = ov.Core()

model = core.read_model(model=model_xml_path)
compiled_model = core.compile_model(model=model, device_name=device.value)

input_layer_ir = compiled_model.input(0)
output_layer_ir = compiled_model.output(0)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import requests
from pathlib import Path

def download_file(url, directory="."):
    Path(directory).mkdir(parents=True, exist_ok=True)
    filename = url.split("/")[-1]
    filepath = Path(directory) / filename
    if not filepath.exists():
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filepath, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    return filepath

image_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/empty_road_mapillary.jpg"
image_filename = download_file(image_url, directory="data")

image = cv2.imread(str(image_filename))
if image is None:
    raise ValueError(f"Image was not loaded properly from path: {image_filename}")

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_h, image_w, _ = image.shape

# Change this to match your model's input shape:
N, C, H, W = (1, 3, 1024, 2048)

# Resize the image
resized_image = cv2.resize(image, (W, H))

# Reshape the image for model input
input_image = np.expand_dims(resized_image.transpose(2, 0, 1), 0)

plt.imshow(rgb_image)
plt.show()

# ... [Your previous code for loading the image and other image processing operations]

blurred_image = cv2.GaussianBlur(rgb_image, (15, 15), 0)
edges_image = cv2.Canny(image, 100, 200)

sepia_filter = np.array([[0.272, 0.534, 0.131],
                         [0.349, 0.686, 0.168],
                         [0.393, 0.769, 0.189]])
sepia_image = cv2.transform(rgb_image, sepia_filter)

fig, ax = plt.subplots(1, 4, figsize=(20, 5))

ax[0].imshow(rgb_image)
ax[0].set_title('Original Image')
ax[0].axis('off')

ax[1].imshow(blurred_image)
ax[1].set_title('Gaussian Blur')
ax[1].axis('off')

ax[2].imshow(edges_image, cmap='gray')
ax[2].set_title('Canny Edge Detection')
ax[2].axis('off')

ax[3].imshow(sepia_image)
ax[3].set_title('Sepia Filter')
ax[3].axis('off')

plt.tight_layout()
plt.show()
